In [ ]:
import pandas as pd
import requests
import os
# カレントディレクトリを.pyと合わせるために以下を実行
from pathlib import Path
if Path.cwd().name == "notebook":
    os.chdir("..")

In [ ]:
# CSVファイルからDataFrameを読み込む
df = pd.read_csv('data/input/user_info_cleansing.csv', dtype={'zipcode': str})

# 欠損値を「N」に置換
df['zipcode'] = df['zipcode'].fillna('N')

# APIキー（PositionStackで取得）
api_key = '89f36561dea6f8feb8bad2a40ed83480'

# 結果を保存するためのリスト
results = []

# バッチサイズ（一度に処理する郵便番号の数）
batch_size = 10

In [ ]:
def batch_geocode(api_key, zipcodes):
    """PositionStack APIを使用して郵便番号のリストをジオコードする"""
    params = {
        'access_key': api_key,
        'query': ','.join(zipcodes),
        'limit': 10
    }
    response = requests.get('http://api.positionstack.com/v1/forward', params=params)
    data = response.json()
    if response.status_code != 200:
        print(f"エラー: {response.status_code, data, params}")
    return data

In [ ]:
# APIリクエストをバッチで実行
# for i in range(0, len(df['zipcode']), batch_size):
for i in range(0, 20, batch_size):
    # 'N'を含まない郵便番号のバッチを作成
    batch = [zipcode for zipcode in df['zipcode'][i:i + batch_size] if zipcode != 'N']
    display(batch)
    if batch:
        res = batch_geocode(api_key, batch)
        print(res)
        results.extend(res)
display('results:',results)

# # APIの結果をDataFrameに追加（'N'を含む行は除外）
# df['latitude'] = [None] * len(df)
# df['longitude'] = [None] * len(df)
# for i, result in enumerate(results):
#     c
#         df.at[i, 'latitude'] = result['data'][0]['latitude']
#         df.at[i, 'longitude'] = result['data'][0]['longitude']

# print(df)

# # 郵便番号の形式をチェックする関数
# def is_valid_zipcode(zipcode):
#     return zipcode.isdigit() and len(zipcode) == 7

# # 郵便番号の形式が正しいかどうかを新しい列に格納
# df['is_valid'] = df['zipcode'].apply(is_valid_zipcode)

In [ ]:
# df.to_csv('data/input/user_info_cleansing_zipcode_translation.csv', index=True)